<a href="https://colab.research.google.com/github/neelsuthar/pytorch/blob/main/NNmodule_improvement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


NN module


Torch.optim module

In [104]:
class Model(nn.Module):

    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,features):
        out=self.linear(features)
        out=self.sigmoid(out)
        return out



In [105]:
features=torch.rand((10,5))
model=Model(features.shape[1])
print(model(features))

tensor([[0.3848],
        [0.4485],
        [0.4198],
        [0.4259],
        [0.5261],
        [0.5389],
        [0.5422],
        [0.4357],
        [0.5199],
        [0.5461]], grad_fn=<SigmoidBackward0>)


In [106]:
model.layer1.weight

AttributeError: 'Model' object has no attribute 'layer1'

In [107]:
model.layer1.bias

AttributeError: 'Model' object has no attribute 'layer1'

In [108]:
!pip install torchinfo

In [109]:
from torchinfo import summary
summary(model,input_size=(10,5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 1]                   6
├─Sigmoid: 1-2                           [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [110]:
#sequnetial container
class Model(nn.Module):

    def __init__(self, num_features):
        super().__init__()
        self.network1 = nn.Sequential(
        nn.Linear(num_features, 3),
        nn.ReLU(),
        nn.Linear(3, 1),
        nn.Sigmoid()
    )

    def forward(self, features):
        out = self.network1(features)
        return out


In [111]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix

In [112]:
#data set and training pipeline

In [113]:
# dataset
df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [114]:
df.shape

(569, 33)

In [115]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)


In [116]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [117]:
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2
)

In [118]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [119]:
X_train

array([[-0.0600293 ,  0.14012664, -0.11748915, ..., -0.86197128,
        -0.64493391, -0.86707296],
       [ 0.62969405,  0.41141486,  0.63805083, ...,  0.66241056,
         0.30439279, -0.12777433],
       [ 0.48148903,  0.29898911,  0.45974008, ...,  0.50291789,
         1.73642799,  1.27167381],
       ...,
       [ 0.29623276,  0.73158384,  0.238303  , ...,  0.58113064,
        -0.38427132, -0.31532332],
       [ 1.78968331,  0.6069379 ,  1.84061173, ...,  2.3524194 ,
         4.25455729,  1.04225608],
       [-0.78680389, -0.20448272, -0.72913651, ...,  0.10265264,
         0.6342436 ,  2.60803207]])

In [120]:
y_train

,diagnosis
438,B
328,M
223,M
14,M
72,M
...,...
44,M
405,B
54,M
323,M


In [121]:
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [190]:
y_train

array([0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,

In [191]:
# tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)



In [192]:
X_train_tensor.shape

torch.Size([455, 30])

In [193]:

#define the model for our NN

In [194]:
class my_simple_nn(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.linear = nn.Linear(input_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.linear(x)
        out = self.sigmoid(out)
        return out


In [195]:
learning_rate=0.01
epochs=250

In [196]:
loss_function=nn.BCELoss()

In [202]:
import torch.nn as nn
model = my_simple_nn(X_train_tensor.shape[1])
#optimizer
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

for epoch in range(epochs):
    # forward
    y_pred = model(X_train_tensor)

    # loss
    loss = loss_function(y_pred, y_train_tensor)

    #clear gradient before back
    optimizer.zero_grad()

    # backward
    loss.backward()

    # update (FIXED)
    # with torch.no_grad():
    #     model.linear.weight -= learning_rate * model.linear.weight.grad
    #     model.linear.bias -= learning_rate * model.linear.bias.grad

    #     model.linear.weight.grad.zero_()
    #     model.linear.bias.grad.zero_()
    optimizer.step()


    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Epoch 1, Loss: 0.6233595609664917
Epoch 2, Loss: 0.6080256700515747
Epoch 3, Loss: 0.5935631990432739
Epoch 4, Loss: 0.5799146890640259
Epoch 5, Loss: 0.5670245885848999
Epoch 6, Loss: 0.5548402070999146
Epoch 7, Loss: 0.5433117747306824
Epoch 8, Loss: 0.5323922038078308
Epoch 9, Loss: 0.5220380425453186
Epoch 10, Loss: 0.5122082829475403
Epoch 11, Loss: 0.5028654932975769
Epoch 12, Loss: 0.49397507309913635
Epoch 13, Loss: 0.4855051040649414
Epoch 14, Loss: 0.4774262607097626
Epoch 15, Loss: 0.4697118401527405
Epoch 16, Loss: 0.46233734488487244
Epoch 17, Loss: 0.4552801251411438
Epoch 18, Loss: 0.4485197365283966
Epoch 19, Loss: 0.4420371651649475
Epoch 20, Loss: 0.43581506609916687
Epoch 21, Loss: 0.4298374652862549
Epoch 22, Loss: 0.4240896701812744
Epoch 23, Loss: 0.41855818033218384
Epoch 24, Loss: 0.41323041915893555
Epoch 25, Loss: 0.4080948829650879
Epoch 26, Loss: 0.4031409025192261
Epoch 27, Loss: 0.3983585238456726
Epoch 28, Loss: 0.39373862743377686
Epoch 29, Loss: 0.38927

In [203]:
model.linear.weight

Parameter containing:
tensor([[ 0.2623,  0.1887,  0.2349,  0.1069,  0.2170,  0.1575,  0.2162,  0.3361,
         -0.0622, -0.0814,  0.0948,  0.0943,  0.1454,  0.1376,  0.0384,  0.0239,
         -0.1052,  0.1282,  0.0543, -0.1853,  0.2212,  0.1949,  0.3585,  0.1635,
          0.0208,  0.0887,  0.2678,  0.4178,  0.1332,  0.0414]],
       requires_grad=True)

In [204]:
print(X_train_tensor.shape)
print(y_train_tensor.shape)


torch.Size([455, 30])
torch.Size([455, 1])


In [205]:
#model evaluate
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred=(y_pred>0.5).float()
  accuracy=y_pred.eq(y_test_tensor).sum()/y_test_tensor.shape[0]
  print(f"accuracy:{accuracy.item()*100}")


accuracy:94.73684430122375
